### TODO
1. Combine any days with multiple data points into the average of all of those

2. Give each entry a 'score' based on how different each value is from the starting one. EX. if Visceral fat was 6 and now it's 5, that's a 1/6 point, if weight was 130 and now its 120, thats 1/13 points, etc

3. Reverse Data in beginning?

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import csv
from matplotlib.ticker import FormatStrFormatter
import pandas as pd
import sklearn
import scipy
import statsmodels.api as sm
COLUMNS = ["Date", "Weight (lbs)", 
           "BMI", "Body Fat %", "Fat-Free Body Weight (lbs)", 
           "Subcutaneous Fat &", "Visceral Fat", "Body Water %", 
          "Skeletal Muscle %", "Muscle Mass (lbs)", "Bone Mass (lbs)", 
           "Protein %", "BMR (kcal)", "Metabolic Age", "Muscle %"]
DATA = []
DATES = []

In [46]:
def parse_data(filename: str):
    global DATA, DATES
    
    #Read in data
    with open(filename, 'r') as file:
        line = csv.reader(file, delimiter=',')
        #Put data in array of arrays
        for i in line:
            entry = []
            for j in i:
                entry.append(j)
            DATA.append(entry) 

    #Set category names
    categories = DATA[0]
    DATA.pop(0)

    #Get rid of remarks column
    for entry in DATA:
        entry.pop(-1)

    #Get rid of data w/ missing values
    i = 0
    while (i < len(DATA)):
        if DATA[i].count("--") > 0:
            DATA.pop(i)
        else:
            i = i+1

    #Date Formatting to DateTime Object
    for i in DATA:
        spl = i[0].split(",")
        date = spl[0]; time = spl[1]
        d = date.split('/'); t = time.split(':')
        i[0] = dt.datetime(int(d[2]), int(d[0]), int(d[1]), int(t[0]), int(t[1]), int(t[2]))
        DATES.append(i[0])

    #Reverse Data to be oldest to newest
    DATA_COPY = DATA.copy()
    for i in range(len(DATA)):
        DATA[i] = DATA_COPY[len(DATA)-i-1]
    DATES_COPY = DATES.copy()
    for i in range(len(DATES)):
        DATES[i] = DATES_COPY[len(DATES)-i-1]

    #Calculate muscle percentages
    for i in DATA:
        i.append(round((float(i[9]) / float(i[1])*100),1))

    #Round Data
    for i in DATA:
        #0: DateTime
        #1: Weight (xxx.x)
        i[1] = round(float(i[1]), 1)
        #2: BMI (xx.x)
        i[2] = round(float(i[2]), 1)
        #3: Body Fat % (xx.x)
        i[3] = round(float(i[3]), 1)
        #4: FatFreeWeight (xxx.x)
        i[4] = round(float(i[4]), 1)
        #5: Subcut Fat % (xx.x)
        i[5] = round(float(i[5]), 1)
        #6: Visceral Fat (x)
        i[6] = round(float(i[6]), 0)
        #7: Water % (xx.x)
        i[7] = round(float(i[7]), 1)
        #8: Skeletal Muscle % (xx.x)
        i[8] = round(float(i[8]), 1)
        #9: Muscle Mass (xxx.x)
        i[9] = round(float(i[9]), 1)
        #10: (x.x)
        i[10] = round(float(i[10]), 1)
        #11: Protein % (xx.x)
        i[11] = round(float(i[11]), 1)
        #12: BMR (xxxx)
        i[12] = round(float(i[12]), 0)
        #13: Metabolic Age (xx)
        i[13] = round(float(i[13]), 0)
        #14: Muscle % (xx.x)
        i[14] = round(float(i[14]), 1)

In [10]:
#PLOT DATA
def plot_data(idx): 
    global START; global DATES; global COLUMNS; global DATA;

    #Get data to plot
    dat = []
    for i in DATA:
        dat.append(i[idx])
    
    fig, ax = plt.subplots()
    
    plt.xlabel("Date")
    plt.xticks(fontsize=7)
    ax.set_xlim([START, dt.date.today()])
    start, end = ax.get_xlim()
    padding = (end-start)/20
    ax.xaxis.set_ticks(np.arange(start, end+padding, (end-start+padding)/13))
    fig.autofmt_xdate()

    plt.ylabel(COLUMNS[idx])
    plt.yticks(fontsize=7)
    ax.set_ylim([min(dat),max(dat)])
    start, end = ax.get_ylim()
    ax.yaxis.set_ticks(np.arange(start-1,end+1,(end-start+2)/13))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    plt.plot(DATES, dat, color='red', marker='o', markersize=4)
    plt.title(COLUMNS[idx] + " vs. Time")
    plt.show()

In [1]:
def print_stats():
    #STATISTICS
    weights = [i[1] for i in DATA]
    days_since = []
    for i in DATES:
        days_since.append(int((i - DATES[0]).days))

    #Derivatives
    #derivative(func, x0, dx, n, args, order)
    #deriv = scipy.misc.derivative(weights, days_since, dx=1.0)
    derivates = []
    for i in range(1, len(weights)-1):
        weight_diff = weights[i]-weights[i-1]
        day_diff = max(days_since[i]-days_since[i-1], 1)
        deriv = round(weight_diff/day_diff, 2)
        derivates.append(deriv)
    print(derivates)

    # np.random.seed(11)
    # before = weights[:-1]
    # after = weights[0:]
    # diff = [days_since[i] - days_since[i-1] for i in range(1, len(days_since))]
    # df = pd.DataFrame({"weight1":before,"weight2":after, "change":after-before})
    # df.describe()

    # df = pd.DataFrame({"days": days_since, "weights": weights})
    # x, y = df.days, df.weights
    # model = sm.OLS(y, x)
    # results = model.fit()
    # model = LinearRegression()
    # model.fit(x, y)
    # slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(df.days, df.weights)
    # print("WEIGHT VS. TIME STATISTICS")
    # print("Slope: ", slope)
    # print("R-Value: ", r_value)
    # print("P-Value: ", p_value)

In [44]:
def run(show_tables: bool):
    global DATA, DATES
    
    # Get Base Data (Highest Weight) (Will be CUTOFF_DATA[-1])
    weights = [i[1] for i in DATA]
    maxIdx = 0
    maxWeight = 0
    for idx, val in enumerate(weights):
        if (val > maxWeight):
            maxWeight = val
            maxIdx = idx
    CUTOFF_DATA = DATA[maxIdx:]

    COLUMNS = ["Date", "Weight (lbs)", 
               "BMI", "Body Fat %", "Fat-Free Body Weight (lbs)", 
               "Subcutaneous Fat %", "Visceral Fat", "Body Water %", 
              "Skeletal Muscle %", "Muscle Mass (lbs)", "Bone Mass (lbs)", 
               "Protein %", "BMR (kcal)", "Metabolic Age", "Muscle %"]
    #Set scores
    scores = [0 for i in range(len(CUTOFF_DATA))]
    BASELINE = CUTOFF_DATA[0]
    for i in range(len(scores)):
        #Weight
        scores[i] += (BASELINE[1]-CUTOFF_DATA[i][1])/BASELINE[1]*2
        #Body Fat %
        scores[i] += (BASELINE[3]-CUTOFF_DATA[i][3])/BASELINE[3]*2
        #Subcutaneous Fat %
        scores[i] += (BASELINE[5]-CUTOFF_DATA[i][5])/BASELINE[5]*1
        #Visceral Fat
        scores[i] += (BASELINE[6]-CUTOFF_DATA[i][6])/BASELINE[6]*.5
        #Protein %
        scores[i] += (CUTOFF_DATA[i][11]-BASELINE[11])/BASELINE[11]*1
        #Metabolic  Age
        scores[i] += (BASELINE[12]-CUTOFF_DATA[i][12])/BASELINE[12]*.5
        #Muscle %
        scores[i] += (CUTOFF_DATA[i][13]-BASELINE[13])/BASELINE[13]*3
    for i in range(len(scores)):
        scores[i] = round(scores[i], 2)

    days_since = []
    for i in range(maxIdx, len(DATES)):
        days_since.append(int((DATES[i] - DATES[0]).days))

    bestScoreIdx = list(idx for idx, val in enumerate(scores) if (val == max(scores)))[0]
    print("High Score: " + str(max(scores)) + " (" + CUTOFF_DATA[bestScoreIdx][0].strftime("%m/%d/%Y") + ")")

    sevenDayTrend = round(sum(scores[-7:-1])/7, 3)
    print("Seven Day Trend: " + str(sevenDayTrend))
    
    print(len(days_since))
    print(len(scores))

    #Plot scores
    fig, ax = plt.subplots()
    plt.xlabel("Days Since")
    plt.xticks(fontsize=7)
    ax.set_xlim([0, days_since[-1]])
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start, end, 30))
    plt.ylabel("Score")
    plt.yticks(fontsize=7)
    ax.set_ylim([min(scores), max(scores)])
    start, end = ax.get_ylim()
    ax.yaxis.set_ticks(np.arange(start,end,.1))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    
    plt.title("Scores vs. Days Since")
    plt.plot(days_since, scores, color='red', marker='o', markersize=4)
    plt.show()

    if (show_tables):

        plot_data(1)  #Weight
        plot_data(3)  #Body Fat %
        plot_data(14) #Muscle Mass %

In [36]:
def main():
    global DATA, DATES
    DATA = []
    DATES = []
    
    filename = input("Filename: ")
    parse_data(filename)
    show_tables_str = input("Do you want to display tables? ")
    show_tables = False
    if (show_tables_str in ["y", "Y", "Yes", "yes"]):
        show_tables = True
    run(show_tables)
    

In [47]:
main()

Filename: Nov9_Data.csv


AttributeError: 'datetime.datetime' object has no attribute 'split'